In [1]:
import pandas as pd
import numpy as np

from scipy.stats import mode
from tqdm.notebook import tqdm

In [2]:
df_1 = pd.read_csv("data/all_purchases.csv", sep=";")
df_2 = pd.read_csv("data/all_suppliers.csv", sep=";")

In [3]:
df_1[:5]

,purchase,region_code,min_publish_date,purchase_name,forsmallbiz,price,customer,okpd2_code,okpd2_names,item_descriptions
0,Закупка_1,2,2021-11-24,Услуги по проведению финансового аудита,f,190000.00,Заказчик_1,69.2,Услуги по проведению финансового аудита,Услуги по проведению финансового аудита
1,Закупка_2,2,2022-11-18,Услуги по проведению финансового аудита,f,271000.00,Заказчик_1,69.2,Услуги по проведению финансового аудита,Услуги по проведению финансового аудита
2,Закупка_3,2,2021-02-02,оказание информационных услуг с использованием...,t,290000.00,Заказчик_2,58.2,Услуги по предоставлению лицензий на право исп...,оказание информационных услуг с использованием...
3,Закупка_4,2,2021-02-02,"Услуги по дизайнерскому оформлению, изготовлен...",t,190809.00,Заказчик_2,18.1,"Услуги печатные прочие, не включенные в другие...","оказание услуг по дизайнерскому оформлению, из..."
4,Закупка_5,2,2021-02-11,поставка сувенирной продукции для проведения м...,t,278263.34,Заказчик_2,26.2,Клавиатуры || Комплектующие и запасные части д...,Комплект беспроводной клавиатуры и беспроводно...


In [4]:
df_2[:5]

,purchase,supplier,is_winner
0,Закупка_3,Поставщик_1,1
1,Закупка_14,Поставщик_2,1
2,Закупка_14,Поставщик_3,0
3,Закупка_5,Поставщик_4,0
4,Закупка_5,Поставщик_2,0


In [5]:
df_1.shape

(621032, 10)

In [6]:
def get_id(string):
    return int(string.split("_")[1])

In [7]:
get_id('Закупка_1')

1

In [8]:
def transform_data(df: pd.DataFrame, labels):
    df["purchase_id"] = df["purchase"].apply(get_id)
    
    try:
        df["customer_id"] = df["customer"].apply(get_id)
    except:
        df["supplier_id"] = df["supplier"].apply(get_id)
    
    res = df.drop(labels=labels, axis=1).set_index("purchase_id")
    res.index.name = None
    
    return res

In [9]:
df_1_tr = transform_data(df_1, labels=["purchase", "purchase_name", "customer", 
                                       "okpd2_names", "item_descriptions"])

In [10]:
df_1_tr[:5]

,region_code,min_publish_date,forsmallbiz,price,okpd2_code,customer_id
1,2,2021-11-24,f,190000.00,69.2,1
2,2,2022-11-18,f,271000.00,69.2,1
3,2,2021-02-02,t,290000.00,58.2,2
4,2,2021-02-02,t,190809.00,18.1,2
5,2,2021-02-11,t,278263.34,26.2,2


In [11]:
df_1_tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 621032 entries, 1 to 621032
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   region_code       621032 non-null  int64  
 1   min_publish_date  621032 non-null  object 
 2   forsmallbiz       621032 non-null  object 
 3   price             621032 non-null  float64
 4   okpd2_code        621032 non-null  object 
 5   customer_id       621032 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 33.2+ MB


In [12]:
df_2_tr = transform_data(df_2, labels=["purchase", "supplier"])
df_2_tr[:5]

,is_winner,supplier_id
3,1,1
14,1,2
14,0,3
5,0,4
5,0,2


In [13]:
df_concat = pd.merge(df_1_tr, df_2_tr, left_index=True, right_index=True)
df_concat[:5]

,region_code,min_publish_date,forsmallbiz,price,okpd2_code,customer_id,is_winner,supplier_id
1,2,2021-11-24,f,190000.0,69.2,1,1,104753
1,2,2021-11-24,f,190000.0,69.2,1,0,104929
1,2,2021-11-24,f,190000.0,69.2,1,0,37544
2,2,2022-11-18,f,271000.0,69.2,1,1,104753
2,2,2022-11-18,f,271000.0,69.2,1,0,11259


In [14]:
df_concat = df_concat.reset_index(names=["purchase_id"])
df_concat[:5]

,purchase_id,region_code,min_publish_date,forsmallbiz,price,okpd2_code,customer_id,is_winner,supplier_id
0,1,2,2021-11-24,f,190000.0,69.2,1,1,104753
1,1,2,2021-11-24,f,190000.0,69.2,1,0,104929
2,1,2,2021-11-24,f,190000.0,69.2,1,0,37544
3,2,2,2022-11-18,f,271000.0,69.2,1,1,104753
4,2,2,2022-11-18,f,271000.0,69.2,1,0,11259


In [15]:
df_concat.supplier_id.nunique()

144142

In [16]:
df_concat.describe().iloc[1:, :]

,purchase_id,region_code,price,customer_id,is_winner,supplier_id
mean,318040.223172,54.925945,4.245324e+06,12166.599483,0.332403,38380.498168
std,179213.203346,25.965663,1.117028e+08,5468.788044,0.471075,33146.592941
min,1.000000,1.000000,1.000000e-02,1.000000,0.000000,1.000000
25%,159883.000000,33.000000,1.352748e+05,9469.000000,0.000000,9582.000000
50%,325650.000000,64.000000,4.657969e+05,11354.000000,0.000000,30514.000000
75%,473669.000000,77.000000,1.566204e+06,16714.000000,1.000000,57338.000000
max,621032.000000,99.000000,5.703527e+10,23363.000000,1.000000,144142.000000


In [17]:
df_concat.describe(include=object)

,min_publish_date,forsmallbiz,okpd2_code
count,1884897,1884897,1884897
unique,722,2,250
top,2021-03-31,t,32.5
freq,7957,1317979,171982


In [18]:
df_concat["date"] = pd.to_datetime(df_concat['min_publish_date'])

In [19]:
df_concat = df_concat.drop(labels=["min_publish_date"], axis=1)

In [20]:
df_concat[:5]

,purchase_id,region_code,forsmallbiz,price,okpd2_code,customer_id,is_winner,supplier_id,date
0,1,2,f,190000.0,69.2,1,1,104753,2021-11-24
1,1,2,f,190000.0,69.2,1,0,104929,2021-11-24
2,1,2,f,190000.0,69.2,1,0,37544,2021-11-24
3,2,2,f,271000.0,69.2,1,1,104753,2022-11-18
4,2,2,f,271000.0,69.2,1,0,11259,2022-11-18


In [21]:
df_concat['is_winner'] = df_concat["is_winner"] + 1

In [22]:
df_concat[:5]

,purchase_id,region_code,forsmallbiz,price,okpd2_code,customer_id,is_winner,supplier_id,date
0,1,2,f,190000.0,69.2,1,2,104753,2021-11-24
1,1,2,f,190000.0,69.2,1,1,104929,2021-11-24
2,1,2,f,190000.0,69.2,1,1,37544,2021-11-24
3,2,2,f,271000.0,69.2,1,2,104753,2022-11-18
4,2,2,f,271000.0,69.2,1,1,11259,2022-11-18


In [30]:
df_train = df_concat[df_concat.date < pd.Timestamp('2022-12-01 00:00:00')]
df_test = df_concat[df_concat.date >= pd.Timestamp('2022-12-01 00:00:00')]

In [54]:
df_test.shape

(43997, 9)

In [26]:
df_concat[['supplier_id', 'okpd2_code']]

,supplier_id,okpd2_code
0,104753,69.2
1,104929,69.2
2,37544,69.2
3,104753,69.2
4,11259,69.2
...,...,...
1884892,17352,drug
1884893,2940,drug
1884894,5493,drug
1884895,700,drug


In [31]:
sup_n_part = df_train.groupby(["supplier_id", "okpd2_code"])["is_winner"].sum()

In [32]:
sup_n_part

supplier_id  okpd2_code
1            58.2           8
             62.0          36
             63.1           4
             63.9           4
2            18.1           1
                           ..
144138       43.9           2
144139       49.4           2
144140       49.4           2
144141       49.4           1
144142       38.1           1
Name: is_winner, Length: 343391, dtype: int64

In [64]:
df_train.okpd2_code.nunique()

250

In [33]:
sup_okpd = pd.DataFrame(columns=df_train.okpd2_code.unique(), 
                        index=df_train.supplier_id.unique(), 
                        data=np.zeros((df_train.supplier_id.nunique(), 
                                       df_train.okpd2_code.nunique()), dtype=np.int8))

In [34]:
sup_n_part = sup_n_part.reset_index()

In [35]:
sup_n_part

,supplier_id,okpd2_code,is_winner
0,1,58.2,8
1,1,62.0,36
2,1,63.1,4
3,1,63.9,4
4,2,18.1,1
...,...,...,...
343386,144138,43.9,2
343387,144139,49.4,2
343388,144140,49.4,2
343389,144141,49.4,1


In [36]:
sup_n_part.iloc[0, 1]

'58.2'

In [37]:
sup_okpd.loc[sup_n_part.iloc[0, 0], sup_n_part.iloc[0, 1]] = sup_n_part.iloc[0, 2]

In [38]:
for i in tqdm(range(sup_n_part.shape[0])):
    sup_okpd.loc[sup_n_part.iloc[i, 0], sup_n_part.iloc[i, 1]] = sup_n_part.iloc[i, 2]

  0%|          | 0/343391 [00:00<?, ?it/s]

In [39]:
sup_okpd

,69.2,58.2,18.1,26.2,17.1,25.9,25.7,32.9,22.2,26.3,...,98.1,02.3,87.2,19.1,50.2,09.9,50.1,46.9,65.2,47.8
104753,39,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
104929,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
37544,200,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11259,390,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
104758,46,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114627,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
114626,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
114625,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
114630,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
sup_okpd.index

Int64Index([104753, 104929,  37544,  11259, 104758,  11260,  39534,      1,
               856, 114633,
            ...
            114620, 114628, 114622, 114623, 114624, 114627, 114626, 114625,
            114630, 114631],
           dtype='int64', length=142661)

In [112]:
sup_okpd.loc[104753].sort_values(ascending=False).index[:20]

Index(['69.2', '52.1', '13.2', '85.2', '72.2', '59.1', '82.2', '35.1', '64.1',
       '23.6', '47.7', '70.2', '02.2', '30.1', '49.2', '82.9', '38.3', '47.9',
       '24.3', '23.9'],
      dtype='object')

In [143]:
sup_okpd.loc[104753]['69.2']

39

In [41]:
dict_sup_okpd = {}
for sup in sup_okpd.index:
    result = sup_okpd.loc[sup].sort_values(ascending=False).index[:10]
    dict_sup_okpd[sup] = list()
    
    for res in result:
        if sup_okpd.loc[sup][res] != 0:
            dict_sup_okpd[sup].append(res)
        else:
            dict_sup_okpd[sup].append('None')

In [42]:
df_final = pd.DataFrame.from_dict(dict_sup_okpd, orient='index')

In [43]:
def get_mean(sup_id):
    return round(df_concat[df_concat['supplier_id'] == sup_id]['price'].mean())

In [44]:
def get_mode(sup_id):
    return mode(df_concat[df_concat['supplier_id'] == sup_id]['region_code'])[0][0]

In [45]:
df_final = df_final.reset_index()

In [46]:
df_final

,index,0,1,2,3,4,5,6,7,8,9
0,104753,69.2,None,None,None,None,None,None,None,None,None
1,104929,69.2,None,None,None,None,None,None,None,None,None
2,37544,69.2,None,None,None,None,None,None,None,None,None
3,11259,69.2,None,None,None,None,None,None,None,None,None
4,104758,69.2,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
142656,114627,38.1,None,None,None,None,None,None,None,None,None
142657,114626,38.1,None,None,None,None,None,None,None,None,None
142658,114625,69.2,None,None,None,None,None,None,None,None,None
142659,114630,17.1,None,None,None,None,None,None,None,None,None


In [47]:
df_final['price'] = df_final['index'].apply(get_mean)

In [48]:
df_final['region'] = df_final['index'].apply(get_mode)

/tmp/ipykernel_16946/832093875.py:2: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  return mode(df_concat[df_concat['supplier_id'] == sup_id]['region_code'])[0][0]


In [60]:
df_final

,index,0,1,2,3,4,5,6,7,8,9,price,region
0,104753,69.2,None,None,None,None,None,None,None,None,None,234807,2
1,104929,69.2,None,None,None,None,None,None,None,None,None,168833,2
2,37544,69.2,None,None,None,None,None,None,None,None,None,449833,77
3,11259,69.2,None,None,None,None,None,None,None,None,None,293767,77
4,104758,69.2,None,None,None,None,None,None,None,None,None,404175,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
142656,114627,38.1,None,None,None,None,None,None,None,None,None,269771458,29
142657,114626,38.1,None,None,None,None,None,None,None,None,None,50041648,29
142658,114625,69.2,None,None,None,None,None,None,None,None,None,150000,2
142659,114630,17.1,None,None,None,None,None,None,None,None,None,261750,33


In [61]:
df_final_2 = df_final[['index', 0, 'price', 'region']]

In [62]:
df_final_2 = df_final_2.rename(columns={0: "okpd"})

In [63]:
df_final_2

,index,okpd,price,region
0,104753,69.2,234807,2
1,104929,69.2,168833,2
2,37544,69.2,449833,77
3,11259,69.2,293767,77
4,104758,69.2,404175,2
...,...,...,...,...
142656,114627,38.1,269771458,29
142657,114626,38.1,50041648,29
142658,114625,69.2,150000,2
142659,114630,17.1,261750,33
